# Monte-Carlo Dropout for uncertainty prediction on MNIST

We implement Monte-Carlo dropout for uncertainty prediction according to https://arxiv.org/pdf/1506.02142.pdf

Therefore we train a model on MNIST using dropout and then at inference time we keep using dropout to estimate the uncertainty of the network by performing several forward passes through the network for each sample. 

# Imports

In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


# Hyperparameters

In [2]:
batch_size = 128
num_classes = 10
epochs = 12
mc_samples = 10 # number of samples for prediciton

# Load and preprocess MNIST

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Build model

In [4]:
def get_model():
    """
    Use only minimalistic model to get some statistics for misclassifications
    """
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    # This dropout layer stays active during testing phase
    model.add(Lambda(lambda x: K.dropout(x, level=0.5)))
    
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    return model

In [5]:
model = get_model()

In [6]:
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 87us/step - loss: 0.3705 - acc: 0.8928 - val_loss: 0.2005 - val_acc: 0.9402
Epoch 2/12
60000/60000 [==============================] - 4s 63us/step - loss: 0.1745 - acc: 0.9491 - val_loss: 0.1378 - val_acc: 0.9616
Epoch 3/12
60000/60000 [==============================] - 4s 63us/step - loss: 0.1375 - acc: 0.9590 - val_loss: 0.1150 - val_acc: 0.9671
Epoch 4/12
60000/60000 [==============================] - 4s 63us/step - loss: 0.1192 - acc: 0.9647 - val_loss: 0.1046 - val_acc: 0.9666
Epoch 5/12
60000/60000 [==============================] - 4s 63us/step - loss: 0.1079 - acc: 0.9681 - val_loss: 0.0974 - val_acc: 0.9699
Epoch 6/12
60000/60000 [==============================] - 4s 64us/step - loss: 0.0989 - acc: 0.9708 - val_loss: 0.0923 - val_acc: 0.9718
Epoch 7/12
60000/60000 [==============================] - 4s 63us/step - loss: 0.0934 - acc: 0.9726 - val_loss: 0.0942 - val_acc

# Evaluate Uncertainty Prediction with Monte Carlo Dropout

Evaluate on test set and compare mean standard deviation of all predictions to mean standard deviation of misclassified samples

In [7]:
def get_predictions_with_uncertainty(model, X):
    
    predictions = []
    for i in range(mc_samples): # can be made more efficient by just forward passing several times through the last layer
        predictions.append(model.predict(X))
    predictions = np.array(predictions)
    
    means = np.mean(predictions, axis=0)
    std = np.std(predictions, axis=0)
    preds = np.argmax(means, axis=1)
    preds_std = np.array([std[i, preds[i]] for i in range(len(preds))])

    return preds, preds_std

In [8]:
preds, stds = get_predictions_with_uncertainty(model, x_test)
labels = np.argmax(y_test, axis=1)

print('Accuracy: ' + str((preds == labels).sum()/len(labels)))

misclassified_mask = labels != preds

print('Average standard deviation of classification: ' + str(np.mean(stds)))
print('Average standard deviation of misclassified samples: ' + str(np.mean(stds[misclassified_mask])))

Accuracy: 0.9802
Average standard deviation of classification: 0.023104193
Average standard deviation of misclassified samples: 0.20123567
